# 01 - Adatfeltárás (Data Exploration)
Ebben a notebookban betöltjük a rendelkezésre álló CSV fájlokat, és vizualizáljuk az árfolyamokat, hogy ellenőrizzük az adatok minőségét.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import json

# Útvonal beállítása
DATA_DIR = "../data" if os.path.exists("../data") else "/app/data"
print(f"Adatok könyvtára: {DATA_DIR}")

# CSV fájlok keresése minden alkönyvtárban
# A glob '**/*.csv' rekurzívan keres, ha a recursive=True be van állítva
csv_files = glob.glob(os.path.join(DATA_DIR, "**", "*.csv"), recursive=True)

print(f"Összesen {len(csv_files)} CSV fájlt találtunk.")

# Opcionális: Lista kiírása (csak az első 5-öt, hogy ne szemetelje tele a kimenetet)
for f in csv_files[:5]:
    # A relatív utat írjuk ki a DATA_DIR-hez képest
    rel_path = os.path.relpath(f, DATA_DIR)
    print(f" - {rel_path}")
if len(csv_files) > 5:
    print(f" ... és további {len(csv_files)-5} db.")

In [ ]:
def plot_sample(file_path):
    """Betölt egy CSV-t és kirajzolja az első 200 gyertyát."""
    try:
        df = pd.read_csv(file_path)
        # Oszlopnevek egységesítése
        df.columns = [c.capitalize() for c in df.columns]
        
        if 'Timestamp' in df.columns:
            df['Timestamp'] = pd.to_datetime(df['Timestamp'])
            df = df.set_index('Timestamp').sort_index()
        
        plt.figure(figsize=(12, 5))
        # Ellenőrzés, hogy van-e adat
        if len(df) > 0:
            cols_to_plot = 'Close' if 'Close' in df.columns else df.columns[0]
            plt.plot(df.index[:200], df[cols_to_plot][:200], label='Close Price', color='black')
            
        plt.title(f"Minta: {os.path.basename(file_path)}")
        plt.xlabel("Idő")
        plt.ylabel("Ár")
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()
        print(f"Fájl: {file_path}")
        print(f"Méret: {len(df)} sor")
    except Exception as e:
        print(f"Hiba a {file_path} betöltésekor: {e}")

# Minta kirajzolása az első 3 talált fájlból
for f in csv_files[:3]:
    print("-" * 50)
    plot_sample(f)

In [ ]:
def find_csv_path_recursive(filename, root_dir):
    """Megkeresi a fájlt a root_dir alatt bárhol."""
    # 1. Próbáljuk megkeresni a fájlt rekurzívan
    matches = glob.glob(os.path.join(root_dir, "**", filename), recursive=True)
    if matches: return matches[0]
    
    # 2. Ha ID prefix van (pl. uuid-filename.csv), keressük a végét
    simple_name = filename.split('-')[-1] if '-' in filename else filename
    # Ez lassabb, de biztosabb: végigiterálunk az összes csv-n
    all_csvs = glob.glob(os.path.join(root_dir, "**", "*.csv"), recursive=True)
    for f in all_csvs:
        if os.path.basename(f).endswith(simple_name):
            return f
    return None

def visualize_all_patterns():
    # 1. Összes labels.json összegyűjtése
    label_files = glob.glob(os.path.join(DATA_DIR, "**", "labels.json"), recursive=True)
    
    all_tasks = []
    for lf in label_files:
        try:
            with open(lf, 'r') as f:
                tasks = json.load(f)
                # Opcionális: megjegyezhetjük, melyik mappából jött
                current_subdir = os.path.dirname(lf)
                for t in tasks: t['_source_dir'] = current_subdir
                all_tasks.extend(tasks)
        except Exception as e:
            print(f"Hiba a {lf} olvasásakor: {e}")

    targets = {
        "Bullish Normal": False, "Bullish Pennant": False, "Bullish Wedge": False,
        "Bearish Normal": False, "Bearish Pennant": False, "Bearish Wedge": False
    }
    
    print(f"Összesen {len(all_tasks)} task feldolgozása a mintákhoz...")

    for task in all_tasks:
        if all(targets.values()): break

        filename = task.get('file_upload')
        if not filename: continue
        
        # A fájlt vagy a labels.json mappájában keressük, vagy globálisan
        source_dir = task.get('_source_dir', DATA_DIR)
        csv_path = find_csv_path_recursive(filename, DATA_DIR) # Globális keresés a biztonság kedvéért
        
        if not csv_path: continue
        
        df = None # Lazy load
        
        for ann in task.get('annotations', []):
            for res in ann.get('result', []):
                val = res.get('value', {})
                labels = val.get('timeserieslabels', [])
                if not labels: continue
                
                original_label = labels[0]
                
                # Címke felismerése
                direction = "Bullish" if "Bullish" in original_label else ("Bearish" if "Bearish" in original_label else None)
                shape = "Normal"
                if "Pennant" in original_label: shape = "Pennant"
                elif "Wedge" in original_label: shape = "Wedge"
                
                if not direction: continue
                target_key = f"{direction} {shape}"
                
                if target_key in targets and not targets[target_key]:
                    if df is None:
                        try:
                            df = pd.read_csv(csv_path)
                            df.columns = [c.capitalize() for c in df.columns]
                            if 'Timestamp' in df.columns:
                                df['Timestamp'] = pd.to_datetime(df['Timestamp'])
                                df = df.set_index('Timestamp').sort_index()
                        except: break

                    start = pd.to_datetime(val['start'])
                    end = pd.to_datetime(val['end'])
                    margin = pd.Timedelta(minutes=30)
                    
                    mask = (df.index >= start - margin) & (df.index <= end + margin)
                    subset = df.loc[mask]
                    
                    if len(subset) > 0:
                        plt.figure(figsize=(10, 4))
                        plt.plot(subset.index, subset['Close'], color='gray', alpha=0.5, label='Context')
                        
                        pat_mask = (subset.index >= start) & (subset.index <= end)
                        plt.plot(subset.loc[pat_mask].index, subset.loc[pat_mask]['Close'], 
                                 color='green' if direction == "Bullish" else 'red', linewidth=2, label=target_key)
                        
                        plt.title(f"{target_key} | {os.path.basename(csv_path)}")
                        plt.legend()
                        plt.show()
                        targets[target_key] = True
                        print(f"✅ {target_key} found.")

visualize_all_patterns()